# - C5i Main Model

In [8]:
import transformers
transformers.logging.set_verbosity_error()


In [4]:
import nltk
nltk.download('words')
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import math
from nltk.corpus import words
import re

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Yaxh\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


## - Datatset loaded

In [5]:
df = pd.read_excel(r"Final Data File_Training.xlsx", sheet_name=1)
print("Dataset loaded successfully!")
print("First 5 rows of the dataset:")
print(df.head())

Dataset loaded successfully!
First 5 rows of the dataset:
   Unique ID          Start Date            End Date  \
0        NaN                 NaT                 NaT   
1        3.0 2024-10-11 09:43:37 2024-10-11 09:53:40   
2        5.0 2024-10-11 09:42:41 2024-10-11 09:55:49   
3        6.0 2024-10-11 09:46:20 2024-10-11 09:56:42   
4        8.0 2024-10-11 09:47:15 2024-10-11 09:57:19   

   Q1. What is your current age? \n(Age)  Q2. What is your gender? \n(Gender)  \
0                                    NaN                                  NaN   
1                                   60.0                                  2.0   
2                                   61.0                                  1.0   
3                                   58.0                                  1.0   
4                                   55.0                                  1.0   

   Q3. Which of the following best describes the area or community in which you live? \n(Urban/Rural)  \
0            

# - Gibberish Detection 

In [ ]:


# Load English words
english_words = set(words.words())

def is_gibberish(text, entropy_threshold=3.46, keyboard_seq_length=4, min_valid_word_ratio=0.3):
    """Detect gibberish text based on entropy, keyboard patterns, and valid word ratio."""
    
    if pd.isna(text) or len(str(text).strip()) < 4:
        return 0  # Not gibberish if empty or too short
    
    text = str(text).lower()
    clean_text = re.sub(r'[^a-z\s]', '', text)
    
    if len(clean_text.strip()) < 3:
        return 0  # Not gibberish if it's too short after cleaning

    tokens = clean_text.split()
    valid_words = sum(1 for word in tokens if word in english_words)
    word_ratio = valid_words / len(tokens) if tokens else 0

    if word_ratio >= min_valid_word_ratio:
        return 0  # Not gibberish if enough words are valid

    # Character entropy calculation
    char_counts = Counter(text)
    text_length = len(text)
    entropy = -sum((count/text_length) * math.log2(count/text_length) for count in char_counts.values())

    # Check for keyboard sequences
    keyboard_rows = ['qwertyuiop', 'asdfghjkl', 'zxcvbnm']
    has_keyboard = any(
        any(text[i:i+keyboard_seq_length] in row or text[i:i+keyboard_seq_length] in row[::-1]
            for row in keyboard_rows)
        for i in range(len(text) - keyboard_seq_length + 1)
    )

    # Check for repeating characters
    has_repeats = any(c * 3 in text for c in set(text))

    # Check for consecutive consonants or vowels
    consonants = 'bcdfghjklmnpqrstvwxyz'
    vowels = 'aeiou'
    has_consecutive = re.search(r'[' + consonants + ']{5}|[' + vowels + ']{4}', clean_text) is not None

    return 1 if (entropy > entropy_threshold or has_keyboard or has_repeats or has_consecutive) else 0


oe_columns = [29, 30]

# Apply gibberish detection and add a new column
df['Gibberish_Flag'] = df.iloc[:, oe_columns].apply(lambda row: max(is_gibberish(row[0]), is_gibberish(row[1])), axis=1)

# Save the updated DataFrame to a new Excel file
df.to_excel("new_yay_updated.xlsx", sheet_name="Sheet1", index=False)

print("Updated dataset saved successfully with 'Gibberish_Flag' column at the end.")


C:\Users\Yaxh\AppData\Local\Temp\ipykernel_17076\2800129156.py:50: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['Gibberish_Flag'] = df.iloc[:, oe_columns].apply(lambda row: max(is_gibberish(row[0]), is_gibberish(row[1])), axis=1)


Updated dataset saved successfully with 'Gibberish_Flag' column at the end.


# - OffTopic Detection

In [ ]:
#### filepath: c:\Users\Yaxh\Desktop\Hackathonn\model.ipynb
import re
import math
import numpy as np
import pandas as pd
from collections import Counter
from rapidfuzz import process, fuzz  # Faster than fuzzywuzzy
from rapidfuzz.utils import default_process



combined_df = pd.read_csv("combined.csv")


valid_entries = set()
for _, row in combined_df.iterrows():
    brand = str(row["Brand"]).strip().lower()
    product = str(row["Product"]).strip().lower()
    valid_entries.update([brand, product])
valid_entries = list(valid_entries)

# Create a regex pattern for substring matching
valid_substrings = re.compile(
    r"\b(" + "|".join(map(re.escape, valid_entries)) + r")\b", flags=re.IGNORECASE
)

# Vectorized preprocessing
def preprocess_column(col):
    return (
        col.astype(str)
        .str.lower()
        .str.replace(r"[^a-z0-9\s]", " ", regex=True)
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )

# Preprocess valid entries once for rapidfuzz
preprocessed_valid = [default_process(entry) for entry in valid_entries]

def rapidfuzz_match(entry):
    processed_entry = default_process(entry)
    # First check exact match
    if processed_entry in preprocessed_valid:
        return 0
    # Then fuzzy match with threshold 85
    result = process.extractOne(processed_entry, preprocessed_valid, scorer=fuzz.ratio, score_cutoff=85)
    return 0 if result else 1

# Main flagging logic (example columns 32, 33, 34)
beer_columns = df.columns[32:35]

for col in beer_columns:
    processed_col = preprocess_column(df[col])
    
    # 1) Exact matches
    exact_mask = processed_col.isin(valid_entries)
    # 2) Substring matches
    substring_mask = processed_col.str.contains(valid_substrings, na=False)
    # 3) Fuzzy matches for remaining
    fuzzy_candidates = processed_col[~(exact_mask | substring_mask)]
    fuzzy_mask = fuzzy_candidates.apply(rapidfuzz_match).replace({0: True, 1: False})
    
    # Combine flags (0 = valid, 1 = flagged)
    final_mask = exact_mask | substring_mask | fuzzy_mask
    df[f"{col}_flag"] = np.where(final_mask, 0, 1)

df.to_excel("flagged_data_optimized.xlsx", index=False)

C:\Users\Yaxh\AppData\Local\Temp\ipykernel_17076\3743088491.py:55: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  substring_mask = processed_col.str.contains(valid_substrings, na=False)
C:\Users\Yaxh\AppData\Local\Temp\ipykernel_17076\3743088491.py:58: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fuzzy_mask = fuzzy_candidates.apply(rapidfuzz_match).replace({0: True, 1: False})
C:\Users\Yaxh\AppData\Local\Temp\ipykernel_17076\3743088491.py:55: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  substring_mask = processed_col.str.contains(valid_substrings, na=False)
C:\Users\Yaxh\AppData\Loc

# - AI detection

In [9]:
from transformers import pipeline
import pandas as pd

# Load AI detection model
detector = pipeline("text-classification", model="roberta-base-openai-detector")

# AI Detection Function
def detect_ai(text):
    if pd.isna(text) or len(text) <= 5:
        return "Human-Written"
    
    result = detector(text)[0]
    return "AI-Generated" if result['label'] == 'LABEL_1' else "Human-Written"

# Apply detection to columns 29 and 30
df["AI_Detection_29"] = df.iloc[:, 29].apply(detect_ai)
df["AI_Detection_30"] = df.iloc[:, 30].apply(detect_ai)

# Check for any AI-generated text
if (df["AI_Detection_29"] == "AI-Generated").any() or (df["AI_Detection_30"] == "AI-Generated").any():
    print("AI-generated text detected!")

# Save the results
df.to_csv("output_with_ai_detection.csv", index=False)

print("✅ AI detection completed! Results saved to output_with_ai_detection.csv")


✅ AI detection completed! Results saved to output_with_ai_detection.csv


# - S-Bert transformer model

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

# Load dataset and clean column names
def load_data_xlsx(file_path, sheet_name="Data Set with Labels Text"):
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Rename columns for clarity
    df.rename(columns={
        "Q16A. What is the most important thing you LIKE about the shown concept}?     This can include anything you would want kept for sure or aspects that might drive you to buy or try it…       Please type a detailed response in the space below":
            "Q16A_Likes",
        "Q16B. What is the most important thing you DISLIKE about the shown concept}?    This can include general concerns, annoyances, or any aspects of the product that need fixed for this to be more appealing to you...     Please type a detailed response in the space below.":
            "Q16B_Dislikes",
        "OE_Quality_Flag": "Quality_Flag"
    }, inplace=True)
    
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

# Prepare combined text feature
def combine_texts(row):
    # If any response is missing, fill with "missing_text"
    text_a = str(row["Q16A_Likes"]) if pd.notna(row["Q16A_Likes"]) else "missing_text"
    text_b = str(row["Q16B_Dislikes"]) if pd.notna(row["Q16B_Dislikes"]) else "missing_text"
    return text_a + " " + text_b

# Train classifier on combined text features using SBERT embeddings
def train_classifier(texts, labels):
    # Load SBERT model and encode text
    sbert_model = SentenceTransformer('all-mpnet-base-v2')
    embeddings = sbert_model.encode(texts)
    
    # Scale embeddings
    scaler = StandardScaler()
    embeddings_scaled = scaler.fit_transform(embeddings)
    
    # Train Logistic Regression with balanced class weights
    clf = LogisticRegression(class_weight="balanced", random_state=42, max_iter=1000, C=1.0)
    clf.fit(embeddings_scaled, labels)
    
    return clf, scaler, sbert_model

# Evaluate classifier on new texts
def evaluate_classifier(clf, scaler, sbert_model, texts, threshold=0.5):
    embeddings = sbert_model.encode(texts)
    embeddings_scaled = scaler.transform(embeddings)
    
    # Get predicted probabilities (for class 1)
    pred_probs = clf.predict_proba(embeddings_scaled)[:, 1]
    predicted_labels = (pred_probs > threshold).astype(int)
    return predicted_labels

# Main function for training on full dataset, testing on balanced set
def main(file_path, sheet_name):
    df = load_data_xlsx(file_path, sheet_name)
    
    # Drop rows with missing Quality_Flag and reset index
    df = df.dropna(subset=["Quality_Flag"]).reset_index(drop=True)
    
    # Convert Quality_Flag to binary labels: assume 1 or '1.0' as Bad, else Good (0)
    df["Quality_Flag_Binary"] = df["Quality_Flag"].apply(lambda x: 1 if x in [1, '1.0'] else 0)
    
    # Combine Q16A and Q16B responses into one feature
    df["Combined_Text"] = df.apply(combine_texts, axis=1)
    
    # Shuffle the dataset
    df = shuffle(df, random_state=42).reset_index(drop=True)
    
    # Split dataset for training and testing
    # Use the whole dataset for training
    train_df = df.copy()

    # Create a balanced test dataset (equal number of 0s and 1s)
    test_good = df[df["Quality_Flag_Binary"] == 0].sample(n=200, random_state=42)  
    test_bad  = df[df["Quality_Flag_Binary"] == 1].sample(n=100, random_state=42)  
    test_df = pd.concat([test_good, test_bad]).reset_index(drop=True)
    
    # Train classifier on combined text from the full dataset
    train_texts = train_df["Combined_Text"].fillna("missing_text").astype(str).tolist()
    train_labels = train_df["Quality_Flag_Binary"].tolist()
    clf, scaler, sbert_model = train_classifier(train_texts, train_labels)
    
    # Evaluate on balanced test set
    test_texts = test_df["Combined_Text"].fillna("missing_text").astype(str).tolist()
    predicted_labels = evaluate_classifier(clf, scaler, sbert_model, test_texts, threshold=0.2)
    
    # Save predictions to test_df
    test_df["Predicted_Values"] = predicted_labels
    
    # Print evaluation metrics
    accuracy = accuracy_score(test_df["Quality_Flag_Binary"], test_df["Predicted_Values"])
    print(f"\n🔹 **Test Accuracy: {accuracy:.4f}**\n")
    print("🔹 **Classification Report:**")
    print(classification_report(test_df["Quality_Flag_Binary"], test_df["Predicted_Values"], zero_division=0))

    
    # Save test results (Quality_Flag and Predicted_Values) to Excel
    output_df = test_df[["Quality_Flag", "Predicted_Values"]]
    output_file = "classified_responses_sbert_balanced_test.xlsx"
    output_df.to_excel(output_file, index=False)
    print(f"\n **Test Results saved to '{output_file}'**")

if __name__ == "__main__":
    xlsx_file = "Final Data File_Training.xlsx"
    sheet_name = "Data Set with Labels Text"
    main(xlsx_file, sheet_name)



🔹 **Test Accuracy: 0.9033**

🔹 **Classification Report:**
              precision    recall  f1-score   support

           0       1.00      0.85      0.92       200
           1       0.78      1.00      0.87       100

    accuracy                           0.90       300
   macro avg       0.89      0.93      0.90       300
weighted avg       0.93      0.90      0.91       300


✅ **Test Results saved to 'classified_responses_sbert_balanced_test.xlsx'**


# - Prediction on test data

In [4]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os

# Load dataset and clean column names
def load_data_xlsx(file_path, sheet_name="Data Set with Labels Text"):
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Rename columns for clarity
    df.rename(columns={
        "Q16A. What is the most important thing you LIKE about the shown concept}?     This can include anything you would want kept for sure or aspects that might drive you to buy or try it…       Please type a detailed response in the space below":
            "Q16A_Likes",
        "Q16B. What is the most important thing you DISLIKE about the shown concept}?    This can include general concerns, annoyances, or any aspects of the product that need fixed for this to be more appealing to you...     Please type a detailed response in the space below.":
            "Q16B_Dislikes",
        "OE_Quality_Flag": "Quality_Flag"
    }, inplace=True)
    
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

# Prepare combined text feature
def combine_texts(row):
    text_a = str(row["Q16A_Likes"]) if pd.notna(row["Q16A_Likes"]) else "missing_text"
    text_b = str(row["Q16B_Dislikes"]) if pd.notna(row["Q16B_Dislikes"]) else "missing_text"
    return text_a + " " + text_b

# Train classifier and save model
def train_and_save_model(file_path, sheet_name, model_path="saved_model"):
    df = load_data_xlsx(file_path, sheet_name)
    
    df = df.dropna(subset=["Quality_Flag"]).reset_index(drop=True)
    df["Quality_Flag_Binary"] = df["Quality_Flag"].apply(lambda x: 1 if x in [1, '1.0'] else 0)
    df["Combined_Text"] = df.apply(combine_texts, axis=1)

    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["Quality_Flag_Binary"])
    
    train_texts = train_df["Combined_Text"].fillna("missing_text").astype(str).tolist()
    train_labels = train_df["Quality_Flag_Binary"].tolist()

    # Load SBERT model and encode text
    sbert_model = SentenceTransformer('all-mpnet-base-v2')
    embeddings = sbert_model.encode(train_texts)

    # Scale embeddings
    scaler = StandardScaler()
    embeddings_scaled = scaler.fit_transform(embeddings)

    # Train classifier
    clf = LogisticRegression(class_weight="balanced", random_state=42, max_iter=1000, C=1.0)
    clf.fit(embeddings_scaled, train_labels)

    # Evaluate on test set
    test_texts = test_df["Combined_Text"].fillna("missing_text").astype(str).tolist()
    test_labels = test_df["Quality_Flag_Binary"].tolist()
    
    test_embeddings = sbert_model.encode(test_texts)
    test_embeddings_scaled = scaler.transform(test_embeddings)
    
    predicted_labels = clf.predict(test_embeddings_scaled)
    pred_probs = clf.predict_proba(test_embeddings_scaled)[:, 1]

    # Save predictions to test_df
    test_df["Predicted_Values"] = predicted_labels
    test_df["Prediction_Probability"] = pred_probs

    # Print evaluation metrics
    accuracy = accuracy_score(test_labels, predicted_labels)
    print(f"\n🔹 **Test Accuracy: {accuracy:.4f}**\n")
    print("🔹 **Classification Report:**")
    print(classification_report(test_labels, predicted_labels, zero_division=0))
    
    # Print confusion matrix
    cm = confusion_matrix(test_labels, predicted_labels)
    print("\n🔹 **Confusion Matrix:**")
    print(f"   True Negative: {cm[0][0]}, False Positive: {cm[0][1]}")
    print(f"   False Negative: {cm[1][0]}, True Positive: {cm[1][1]}")

    # Save test results to Excel
    output_df = test_df[["Quality_Flag_Binary", "Predicted_Values", "Prediction_Probability", "Q16A_Likes", "Q16B_Dislikes"]]
    output_file = "classifier_results_80_20_split.xlsx"
    output_df.to_excel(output_file, index=False)
    print(f"\n🔹 **Test Results saved to '{output_file}'**")

    # Create directory for model saving
    os.makedirs(model_path, exist_ok=True)

    # Save model components
    joblib.dump(clf, f"{model_path}/classifier.pkl")
    joblib.dump(scaler, f"{model_path}/scaler.pkl")
    sbert_model.save(f"{model_path}/sbert_model")

    print(f"\n🔹 Model saved to '{model_path}'")

# Load model and make predictions on new data
def load_model_and_predict(input_file, output_file, model_path="saved_model", sheet_name="Data Set with Labels Text"):
    df = load_data_xlsx(input_file, sheet_name)
    
    df["Combined_Text"] = df.apply(combine_texts, axis=1)

    # Load model components
    clf = joblib.load(f"{model_path}/classifier.pkl")
    scaler = joblib.load(f"{model_path}/scaler.pkl")
    sbert_model = SentenceTransformer(f"{model_path}/sbert_model")

    # Encode and scale text
    texts = df["Combined_Text"].fillna("missing_text").astype(str).tolist()
    embeddings = sbert_model.encode(texts)
    embeddings_scaled = scaler.transform(embeddings)

    # Make predictions
    predicted_labels = clf.predict(embeddings_scaled)

    # Save results to .xlsx
    result_df = pd.DataFrame({
        "Predicted_Quality_Flag": predicted_labels  # Output column
    })
    result_df.to_excel(output_file, index=False)
    
    print(f"\n🔹 Predictions saved to '{output_file}'")

# Example usage:
if __name__ == "__main__":
    train_file = "Final Data File_Training.xlsx"
    new_data_file = "Final Data File_Test.xlsx"
    output_predictions_file = "Predicted_Results.xlsx"

    # Train and save the model
    train_and_save_model(train_file, "Data Set with Labels Text")

    # Load model and make predictions on new dataset
    load_model_and_predict(new_data_file, output_predictions_file)



🔹 **Test Accuracy: 0.8239**

🔹 **Classification Report:**
              precision    recall  f1-score   support

           0       0.93      0.87      0.90       424
           1       0.15      0.28      0.20        36

    accuracy                           0.82       460
   macro avg       0.54      0.57      0.55       460
weighted avg       0.87      0.82      0.85       460


🔹 **Confusion Matrix:**
   True Negative: 369, False Positive: 55
   False Negative: 26, True Positive: 10

🔹 **Test Results saved to 'classifier_results_80_20_split.xlsx'**

🔹 Model saved to 'saved_model'

🔹 Predictions saved to 'Predicted_Results.xlsx'


In [ ]:

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load dataset and clean column names
def load_data_xlsx(file_path, sheet_name="Data Set with Labels Text"):
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Rename columns for clarity
    df.rename(columns={
        "Q16A. What is the most important thing you LIKE about the shown concept}?     This can include anything you would want kept for sure or aspects that might drive you to buy or try it…       Please type a detailed response in the space below":
            "Q16A_Likes",
        "Q16B. What is the most important thing you DISLIKE about the shown concept}?    This can include general concerns, annoyances, or any aspects of the product that need fixed for this to be more appealing to you...     Please type a detailed response in the space below.":
            "Q16B_Dislikes",
        "OE_Quality_Flag": "Quality_Flag"
    }, inplace=True)
    
    # Get beer preference columns (Q18_1, Q18_2, Q18_3)
    beer_preference_columns = [col for col in df.columns if col.startswith('Q18_')]
    print(f"Found {len(beer_preference_columns)} beer preference columns: {beer_preference_columns}")
    
    # Remove unnamed columns
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    
    return df, beer_preference_columns

# Prepare combined text feature
def combine_texts(row):
    # If any response is missing, fill with "missing_text"
    text_a = str(row["Q16A_Likes"]) if pd.notna(row["Q16A_Likes"]) else "missing_text"
    text_b = str(row["Q16B_Dislikes"]) if pd.notna(row["Q16B_Dislikes"]) else "missing_text"
    return text_a + " " + text_b

# Train classifier on combined text features using SBERT embeddings
def train_classifier(texts, labels):
    # Load SBERT model and encode text
    sbert_model = SentenceTransformer('all-mpnet-base-v2')
    embeddings = sbert_model.encode(texts)
    
    # Scale embeddings
    scaler = StandardScaler()
    embeddings_scaled = scaler.fit_transform(embeddings)
    
    # Train Logistic Regression with balanced class weights
    clf = LogisticRegression(class_weight="balanced", random_state=42, max_iter=1000, C=1.0)
    clf.fit(embeddings_scaled, labels)
    
    return clf, scaler, sbert_model

# Evaluate classifier on new texts
def evaluate_classifier(clf, scaler, sbert_model, texts, threshold=0.5):
    embeddings = sbert_model.encode(texts)
    embeddings_scaled = scaler.transform(embeddings)
    
    # Get predicted probabilities (for class 1)
    pred_probs = clf.predict_proba(embeddings_scaled)[:, 1]
    predicted_labels = (pred_probs > threshold).astype(int)
    return predicted_labels, pred_probs

# Analyze beer preferences by prediction group
def analyze_beer_preferences(df, beer_preference_columns):
    results = {}
    
    # Skip analysis if no preference columns or if they're not in the dataframe
    if not beer_preference_columns or not any(col in df.columns for col in beer_preference_columns):
        print("\n🔹 No beer preference columns found for analysis")
        return results
    
    # Overall analysis
    print("\n🔹 **Beer Preference Analysis**")
    
    # Count the most common products for each preference column
    print("\n📊 Most Common Products:")
    for col in beer_preference_columns:
        if col in df.columns:
            # Get top 5 most common products
            top_products = df[col].value_counts().head(5)
            total_responses = len(df[col].dropna())
            
            print(f"\n   {col}:")
            for product, count in top_products.items():
                if pd.notna(product) and product != '' and str(product).strip().lower() != 'none':
                    pct = (count / total_responses) * 100
                    print(f"     {product}: {count} responses ({pct:.1f}%)")
    
    # By True Label (Actual Quality)
    print("\n📊 Products by Quality Label:")
    for label in [0, 1]:
        label_df = df[df["Quality_Flag_Binary"] == label]
        print(f"\n   Quality = {label} ({'Good' if label == 0 else 'Bad'}):")
        
        for col in beer_preference_columns:
            if col in df.columns:
                # Get top 3 products for this group
                top_products = label_df[col].value_counts().head(3)
                total_responses = len(label_df[col].dropna())
                
                if total_responses > 0:
                    print(f"   - {col}:")
                    for product, count in top_products.items():
                        if pd.notna(product) and product != '' and str(product).strip().lower() != 'none':
                            pct = (count / total_responses) * 100
                            print(f"     {product}: {count} responses ({pct:.1f}%)")
    
    # Accuracy by most common products
    print("\n📊 Accuracy by Product Preference:")
    for col in beer_preference_columns:
        if col in df.columns:
            print(f"\n   {col}:")
            # Get products with at least 10 responses
            product_counts = df[col].value_counts()
            common_products = product_counts[product_counts >= 10].index.tolist()
            
            for product in common_products:
                if pd.notna(product) and product != '' and str(product).strip().lower() != 'none':
                    product_df = df[df[col] == product]
                    if len(product_df) > 0:
                        acc = accuracy_score(product_df["Quality_Flag_Binary"], product_df["Predicted_Values"])
                        # Calculate percentage of bad responses for this product
                        bad_pct = (product_df["Quality_Flag_Binary"].sum() / len(product_df)) * 100
                        print(f"     {product} (n={len(product_df)}): Accuracy={acc:.4f}, Bad Responses={bad_pct:.1f}%")
    
    return results

# Main function with 80/20 train/test split
def main(file_path, sheet_name):
    df, beer_preference_columns = load_data_xlsx(file_path, sheet_name)
    
    # Drop rows with missing Quality_Flag and reset index
    df = df.dropna(subset=["Quality_Flag"]).reset_index(drop=True)
    
    # Convert Quality_Flag to binary labels: assume 1 or '1.0' as Bad, else Good (0)
    df["Quality_Flag_Binary"] = df["Quality_Flag"].apply(lambda x: 1 if x in [1, '1.0'] else 0)
    
    # Combine Q16A and Q16B responses into one feature
    df["Combined_Text"] = df.apply(combine_texts, axis=1)
    
    # Count class distribution
    class_counts = df["Quality_Flag_Binary"].value_counts()
    print(f"\n🔹 Class distribution in dataset:")
    print(f"   - Good responses (0): {class_counts.get(0, 0)}")
    print(f"   - Bad responses (1): {class_counts.get(1, 0)}")
    
    # Split dataset into 80% training and 20% testing
    train_df, test_df = train_test_split(
        df, 
        test_size=0.2, 
        random_state=42, 
        stratify=df["Quality_Flag_Binary"]  # Maintain class distribution
    )
    
    print(f"\n🔹 Training set size: {len(train_df)}")
    print(f"🔹 Test set size: {len(test_df)}")
    
    # Train classifier on combined text from the training set
    train_texts = train_df["Combined_Text"].fillna("missing_text").astype(str).tolist()
    train_labels = train_df["Quality_Flag_Binary"].tolist()
    clf, scaler, sbert_model = train_classifier(train_texts, train_labels)
    
    # Evaluate on test set with different thresholds
    thresholds = [0.3, 0.5, 0.7, 0.9]
    best_threshold = 0.5
    best_f1 = 0
    
    print("\n🔹 **Threshold Analysis:**")
    for threshold in thresholds:
        test_texts = test_df["Combined_Text"].fillna("missing_text").astype(str).tolist()
        predicted_labels, pred_probs = evaluate_classifier(clf, scaler, sbert_model, test_texts, threshold=threshold)
        
        # Calculate metrics
        test_df["Predicted_Values"] = predicted_labels
        test_df["Prediction_Probability"] = pred_probs
        
        # Print evaluation metrics
        accuracy = accuracy_score(test_df["Quality_Flag_Binary"], test_df["Predicted_Values"])
        report = classification_report(test_df["Quality_Flag_Binary"], test_df["Predicted_Values"], 
                                      zero_division=0, output_dict=True)
        
        # Get F1 score for the positive class (bad responses) if it exists in the report
        f1_score = report.get('1', {}).get('f1-score', 0)
        
        if f1_score > best_f1:
            best_f1 = f1_score
            best_threshold = threshold
        
        print(f"\n   Threshold: {threshold}")
        print(f"   Accuracy: {accuracy:.4f}")
        print(f"   F1 Score (bad responses): {f1_score:.4f}")
        
        # Confusion matrix
        cm = confusion_matrix(test_df["Quality_Flag_Binary"], test_df["Predicted_Values"])
        print(f"   True Negative: {cm[0][0]}, False Positive: {cm[0][1]}")
        print(f"   False Negative: {cm[1][0]}, True Positive: {cm[1][1]}")
    
    # Use the best threshold for final evaluation
    print(f"\n🔹 **Using best threshold: {best_threshold}**")
    test_texts = test_df["Combined_Text"].fillna("missing_text").astype(str).tolist()
    predicted_labels, pred_probs = evaluate_classifier(clf, scaler, sbert_model, test_texts, threshold=best_threshold)
    
    # Save predictions to test_df
    test_df["Predicted_Values"] = predicted_labels
    test_df["Prediction_Probability"] = pred_probs
    
    # Print evaluation metrics
    accuracy = accuracy_score(test_df["Quality_Flag_Binary"], test_df["Predicted_Values"])
    print(f"\n🔹 **Final Test Accuracy: {accuracy:.4f}**\n")
    print("🔹 **Classification Report:**")
    print(classification_report(test_df["Quality_Flag_Binary"], test_df["Predicted_Values"], zero_division=0))
    
    # Print confusion matrix
    cm = confusion_matrix(test_df["Quality_Flag_Binary"], test_df["Predicted_Values"])
    print("\n🔹 **Confusion Matrix:**")
    print(f"   True Negative: {cm[0][0]}, False Positive: {cm[0][1]}")
    print(f"   False Negative: {cm[1][0]}, True Positive: {cm[1][1]}")
    
    # Analyze beer preferences
    analyze_beer_preferences(test_df, beer_preference_columns)
    
    # Columns to save
    output_columns = ["Quality_Flag_Binary", "Predicted_Values", "Prediction_Probability", 
                      "Q16A_Likes", "Q16B_Dislikes"]
    
    # Add beer preference columns to output
    for col in beer_preference_columns:
        if col in test_df.columns:
            output_columns.append(col)
    
    # Save test results to Excel with detailed analysis
    output_df = test_df[output_columns]
    output_file = "classifier_results_with_beer_preferences.xlsx"
    output_df.to_excel(output_file, index=False)
    print(f"\n🔹 **Test Results saved to '{output_file}'**")
    
    # Create additional sheet with product-specific analysis
    if beer_preference_columns:
        with pd.ExcelWriter(output_file, engine='openpyxl', mode='a') as writer:
            # Top products summary
            product_summary = []
            for col in beer_preference_columns:
                if col in df.columns:
                    top_prods = df[col].value_counts().head(10).reset_index()
                    top_prods.columns = ['Product', 'Count']
                    top_prods['Column'] = col
                    product_summary.append(top_prods)
            
            if product_summary:
                pd.concat(product_summary).to_excel(writer, sheet_name='Top Products', index=False)
            
            # Product accuracy summary
            product_accuracy = []
            for col in beer_preference_columns:
                if col in test_df.columns:
                    product_counts = test_df[col].value_counts()
                    common_products = product_counts[product_counts >= 5].index.tolist()
                    
                    for product in common_products:
                        if pd.notna(product) and product != '' and str(product).strip().lower() != 'none':
                            product_df = test_df[test_df[col] == product]
                            if len(product_df) > 0:
                                acc = accuracy_score(product_df["Quality_Flag_Binary"], product_df["Predicted_Values"])
                                bad_pct = (product_df["Quality_Flag_Binary"].sum() / len(product_df)) * 100
                                
                                product_accuracy.append({
                                    'Column': col,
                                    'Product': product,
                                    'Count': len(product_df),
                                    'Accuracy': acc,
                                    'Bad_Response_Pct': bad_pct
                                })
            
            if product_accuracy:
                pd.DataFrame(product_accuracy).to_excel(writer, sheet_name='Product Accuracy', index=False)

if __name__ == "__main__":
    xlsx_file = "Final Data File_Training.xlsx"
    sheet_name = "Data Set with Labels Text"
    main(xlsx_file, sheet_name)

Found 3 beer preference columns: ['Q18_1 What specific product that you are currently using would the shown product replace?\n Please type in ONE specific brand or product per space provided.', 'Q18_2 What specific product that you are currently using would the shown concept replace?\n Please type in ONE specific brand or product per space provided.', 'Q18_3 What specific product that you are currently using would the shown concept replace?\n Please type in ONE specific brand or product per space provided.']

🔹 Class distribution in dataset:
   - Good responses (0): 2119
   - Bad responses (1): 180

🔹 Training set size: 1839
🔹 Test set size: 460

🔹 **Threshold Analysis:**
   Threshold: 0.3
   Accuracy: 0.7826
   F1 Score (bad responses): 0.1667
   True Negative: 350, False Positive: 74
   False Negative: 26, True Positive: 10

   Threshold: 0.5
   Accuracy: 0.8217
   F1 Score (bad responses): 0.1961
   True Negative: 368, False Positive: 56
   False Negative: 26, True Positive: 10

   